In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.widgets as wid
import scipy.signal as sig
import scipy.signal.windows as win
import scipy 
import scipy.io
from scipy.signal import lfilter

import numpy.random as rand

import util

## 1. Étude Théorique : Analyse du mélange de deux processus

### Montrer que

$$
S_Z(\nu) = S_Y(\nu) + S_T(\nu)
$$

On a $Z = h \star X + Y$

$h$ est stable, $X$ est SSL donc $T$ est SSL par le théorème de filtrage des processus SSL. Par le résultat admis, on a $T$ et $Y$ indépendants.

On montre d'abord que $Z$ est SSL.

1. On a, pour tout $n$,
$$
E(|T_n + Y_n|^2) = E(|T_n|^2 + 2 |T_nY_n| + |Y_n|^2)
$$
$$
= E(|T_n|^2) + 2E(|T_nY_n|) + E(|Y_n|^2)
$$
Avec $T_n$ et $Y_n$ indépendants et SSL:
$$
= E(|T_n|^2) + E(|T_n|)E(|Y_n|) + E(|Y_n|^2) < +\infty
$$

2. On a bien :
$$
E(Z_n) = E(T_n) + E(Y_n) = m_T + m_Y = m_Z, \forall n
$$

3. On a :
$$
R_Z(k) = R_T(k) + R_Y(k) + E(T_{n+k}^{c}\bar{Y_{n}^c}) + E(\bar{T_{n}^c}Y_{n+k}^{c})
$$
Avec les termes de covariance nuls car $T$ et $Y$ sont indépendants :
$$
R_Z(k) = R_T(k) + R_Y(k)
$$
Ne dépend que de $k$, a fortiori :
$$
S_Z(\nu) = S_T(\nu) + S_Y(\nu)
$$

### Montrer que

Quelque soit la suite sommable $h^1$ et on notant $Z^1$ le processus $Z^1 = Z − h^1 ∗ X$ on a : 

$$
R_{Z^1}(0) \geq R_Y(0)
$$
Et si $R_{Z^1}(0) \geq R_Y(0)$ alors $h^1 = h$.


Par le calcul on a :
$$
R_{Z^1}(0) = R_Y(0) + R_{(h - h^1) ∗ X_n}(0) \geq 0
$$
Soit :
$$
R_{Z^1}(0) \geq R_Y(0)
$$
Et si $h^1 = h$ on obtient :
$$
R_{Z^1}(0) = R_Y(0) + R_{\tilde{0} ∗ X_n}(0) = R_Y(0)
$$

### 1.1 Première application : Identifier un canal

In [17]:
def part_1_1(x, h, n, sigma):
    h = np.array(h)
    y = sigma * rand.randn(n)
    z = lfilter(h, [1], x) + y
    # ici on connaît p, on peut l'écrire
    p = len(h) - 1
    h_estimated = util.moindres_carres(p, x, z)
    h_err = util.norm(h_estimated - h) / util.norm(h)
    return h_estimated, h_err

def print_part_1_1(x, h, n, sigma):
    h_estimated, h_err = part_1_1(x, h, n, sigma)
    print(f"h = {h}")
    print(f"h estimé = {h_estimated}")
    print(f"Erreur = {h_err}")
    return x, h_estimated


In [19]:
# Donnés
h = [1, 0.5]
#
n = 1000
x = rand.randn(n)
sigmas = [0.1, 1, 10]
for sigma in sigmas:
    print(f"sigma = {sigma}")
    print_part_1_1(x, h, n, sigma)

sigma = 0.1
h = [1, 0.5]
h estimé = [1.00006842 0.49429886]
Erreur = 0.005099619588684389
sigma = 1
h = [1, 0.5]
h estimé = [1.02414696 0.48781803]
Erreur = 0.024190511425913233
sigma = 10
h = [1, 0.5]
h estimé = [1.09945633 0.69074004]
Erreur = 0.19240233909325208


L'erreur semble augmenter linéairement avec $\sigma$.

In [20]:
h = [1, 0.5]
#
ns = np.power(2, np.arange(10, 15))  
sigma = 10
for n in ns:
    x = rand.randn(n)
    print(f"n = {n}")
    print_part_1_1(x, h, n, sigma)

n = 1024
h = [1, 0.5]
h estimé = [0.5343664  0.40862675]
Erreur = 0.42441839538099474
n = 2048
h = [1, 0.5]
h estimé = [1.42265195 0.79836338]
Erreur = 0.46273567280796213
n = 4096
h = [1, 0.5]
h estimé = [1.07948675 0.29668384]
Erreur = 0.19525491991640387
n = 8192
h = [1, 0.5]
h estimé = [0.99326541 0.43430459]
Erreur = 0.05906769873771628
n = 16384
h = [1, 0.5]
h estimé = [1.01277303 0.48258328]
Erreur = 0.01931822941823464


On note qu'en moyenne pour $\sigma = 10$, il faut $N = 2^{14}$ pour obtenir une erreur relative $e \leq 0.1$.

On déterminera la longueur de $h$ empiriquement.

Si $h$ trop court, ça ne va pas fonctionner, si
$h$ trop long les derniers coefficients seront nuls et on pourra réduire. On choisira $p$ tel que $h_n = 0, \forall n \geq p$.

In [25]:
h = [1, 0.5]
#
ns = np.power(2, np.arange(10, 15))  
sigma = 1
for n in ns:
    x = np.zeros(n)
    x[n//2] = 1
    print(f"n = {n}")
    print_part_1_1(x, h, n, sigma)

n = 1024
h = [1, 0.5]
h estimé = [2.05769126 1.24623957]
Erreur = 1.1577855758127114
n = 2048
h = [1, 0.5]
h estimé = [-0.24558306  0.76583119]
Erreur = 1.1391728126739546
n = 4096
h = [1, 0.5]
h estimé = [0.34104511 2.60574057]
Erreur = 1.973497382206029
n = 8192
h = [1, 0.5]
h estimé = [2.30248725 0.30818466]
Erreur = 1.1775452995365283
n = 16384
h = [1, 0.5]
h estimé = [ 1.92842089 -0.40217315]
Erreur = 1.1578883358970224


Prendre $X = \delta[0]$ ne peut pas fonctionner pas car l'impulsion est absorbée dans le régime transitoire de $h$, et ne permet pas de le déterminer.
Dans le cas plus général de $X = \delta[a]$, la réponse impulsionnelle est absorbée par $X$, et on a le même problème.

### 1.2 Seconde application : Suppression d'un son qui en parasite un autre

In [38]:
def last_pred(pred, arr, axis, invalid_val=-1):
    # trouver les indices des dernières valeurs vérifiant pred
    # source : https://stackoverflow.com/a/47269413
    mask = pred(arr)
    val = arr.shape[axis] - np.flip(mask, axis=axis).argmax(axis=axis)
    return np.where(mask.any(axis=axis), val, invalid_val)

In [48]:
f_e = 44100
mat = scipy.io.loadmat('parasite.mat')
x_par = mat['Xparasite'].reshape(-1)
z_par = mat['Zparasite'].reshape(-1)

p_prior = 20 # assez grand
h_estimated_prior = util.moindres_carres(p_prior, x_par, z_par)
# p est le dernier indice avant h(n) = 0 stationnaire
precision = 0.01
p = len(h_estimated_prior) - 1 -  last_pred(lambda arr: np.abs(arr) < precision, np.flip(h_estimated_prior, axis=0), 0)
# puis on réitère
h_estimated = util.moindres_carres(p, x_par, z_par)
print(f"p = {p}")
p_err = (util.norm(h_estimated_prior) - util.norm(h_estimated))/util.norm(h_estimated)
print(f"Erreur par réduction de p: {p_err}")
y = z_par - lfilter(h_estimated, [1], x_par)

# util.play(z_par, f_e)
util.play(y_par, f_e)


p = 3
Erreur sur p: -5.537056913862415e-06



/tmp/tmpamdj24a8SON_TP.wav:

 File Size: 198k      Bit Rate: 706k
  Encoding: Signed PCM    
  Channels: 1 @ 16-bit   
Samplerate: 44100Hz      
Replaygain: off         
  Duration: 00:00:02.25  

In:100%  00:00:02.25 [00:00:00.00] Out:99.2k [ -====|====- ] Hd:0.0 Clip:0    
Done.


## 2 Le modèle auto-régressif pour la synthèse de parole

$\epsilon_n$ est tel que :
$$
X_n + h_1X_{n-1} + ... + h_p X_{n-p} = \epsilon_n
$$

## 3 Estimation de la densité spectrale de puissance : périodogramme